#### 사용자 정의 모델 클래스
 - 부모클래스 : nn.Module
 - 필수 오버라이딩
    * __init__() : 모델층 구성 즉, 설계
    * forward() : 순방향 학습 진행 코드 구현

- 동적 모델
    * container 모듈 중 nn.ModelList() 사용해서 동적으로 Layer 추가
        - forward 기능 미 제공
        - layer 인스턴스 요소 사이ㅏ에 연관성없음
        - layer 인스턴스 요소는 인덱싱으로 접근

In [1]:
## 모듈 로딩
import torch                                # 텐서 관련 모듈
import torch.nn as nn                       # 인공신경망 관련 모듈
import torch.nn.functional as F             # 인공신경망 관련 함수들 모듈 ( 손실함수, 활성화함수 등등 )
import torch.optim as optim                 # 최적화 관련 모듈 (가중치, 절편 빠르게 찾아주는 알고리즘)
from torchinfo import summary               # 모델 구조 및 정보 관련 모듈
import torchmetrics as metrics              
from torchmetrics.regression import *       # 회귀 성능 지표 관련 모듈
from torchmetrics.classification import *   # 분류 성능 지표 관련 모듈

In [2]:
# 랜덤 고정
torch.manual_seed(1)

# 텐서 저장 및 실행 위치 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

- 모델 설계 - 동적 모델 <hr>
    * 목표 : 은닉층의 개수가 동적인 모듈
    * 조건
        * 입력층과 출력층 개수 동적 => 입력층의 입력값, 출력층의 출력값
        * 은닉층의 개수 동적 + 퍼셉트론 개수 고정 => 은닉층의 개수, 은닉층 마다 퍼셉트론 수


In [3]:
# -------------------------------------------------------------
# 모델이름 : DynamicModel
# 부모클래스 : nn.Model
# 매개변수 : in_in, out_out, *h_ins=[], h_outs=[]
# -------------------------------------------------------------

In [13]:
class DynamicModel(nn.Module):
    # 모델 구조 설계 함수 즉, 생성자 메서드
    def __init__(self,in_in, in_out, out_out, h_ins=[], h_outs=[]):
        super().__init__()

        self.in_layer = nn.Linear(in_in, h_ins[0] if len(h_ins) else in_out)

        for idx in range(len(h_ins)):          
            self.h_layers.append((nn.Linear(h_ins[idx], h_outs[idx])))
            
            # nn.Linear(h_in, h_inout) for _ in range(h_cnt)]
        
        self.out_layer = nn.Linear(h_outs[-1] if len(h_outs) else in_out, out_out)

    def forward(self, x):
        # 입력층
        y = self.in_layer(x)        # y = w1x1 + w2x2 + w3x3 + b
        y = F.relu(y)               # 0 <= y 

        # 은닉층
        for linear in self.h_layers:
            y = linear(y)
            y = F.relu(y)

        # 출력층
        return self.out_layer(y)


In [14]:
# 모델 인스턴스 생성

# h_ins, h_outs = [30,50,70], [50,70,30]

h_ins_, h_outs_ = range(10,101,20), range(30,121,30)

# m1 = DynamicModel(in_in = 3, in_out = 5, out_out=2, h_ins = h_ins, h_outs=h_outs)
m1 = DynamicModel(in_in = 3, in_out = 5, out_out=2, h_ins = h_ins_, h_outs=h_outs_)

AttributeError: 'DynamicModel' object has no attribute 'h_layers'

In [10]:
# 모델 구조 확인
print(m1)
for name, param in m1.named_parameters():
    print(name, param.shape)

DynamicModel(
  (in_layer): Linear(in_features=3, out_features=5, bias=True)
  (out_layer): Linear(in_features=5, out_features=2, bias=True)
)
in_layer.weight torch.Size([5, 3])
in_layer.bias torch.Size([5])
out_layer.weight torch.Size([2, 5])
out_layer.bias torch.Size([2])


- 학습 진행

In [7]:
## 임시 데이터 생성
dataTS = torch.FloatTensor([[1,3,5],[2,4,6],[3,5,7],[4,6,8]])   # 2D (4,3)
targetTS = torch.FloatTensor([[7,9],[8,10],[9,11],[10,12]])

In [8]:
# 모델 학습
pre_y = m1(dataTS)

print(pre_y, pre_y.shape, sep='\n')

AttributeError: 'DynamicModel' object has no attribute 'h_layers'